Data Prep and Imports

In [69]:
import pandas as pd
import numpy as np

leak_data_raw =  pd.read_csv("Gasleak Data Sets/sensor_readings.csv")
leak_data_raw.columns = ["temp","Time","Sensor 1", "Sensor 2", "Sensor 3", "Sensor 4", "Sensor 5", "Sensor 6", "Sensor 7", "Sensor 8", "Sensor 9", "Sensor 10", "Sensor 11", "Sensor 12", "Sensor 13", "Sensor 14", "Sensor 15", "Sensor 16", "Sensor 17", "Sensor 18", "Sensor 19", "Sensor 20", "Sensor 21", "Sensor 22", "Sensor 23", "Sensor 24"]

weather_df = pd.read_csv("Gasleak Data Sets/weather_data.csv")
columns_to_average = ['Barometric_Pressure', 'Humidity', 'Temperature', 'Wind_Direction', 'Wind_Speed']
for col in columns_to_average:
    weather_df[col] = weather_df.groupby('timestamp')[col].transform('mean')
weather_df = weather_df.drop_duplicates()
weather_df['timestamp'] = pd.to_datetime(weather_df['timestamp']).astype(int) // 10**9
weather_df = weather_df.rename(columns={'timestamp': 'tTime'})

leak_rate_df = pd.read_csv("Gasleak Data Sets/leak_locations_and_rate.csv")
leak_rate_df =  leak_rate_df[['LeakRate', 'tStart', 'tEnd', 'Duration']]
leak_rate_df['LeakRate'] = leak_rate_df.groupby('tStart')['LeakRate'].transform('mean')
leak_rate_df = leak_rate_df.drop_duplicates()


/var/folders/qt/r87857tx74n48x9wj7f9wljw0000gn/T/ipykernel_26112/2984164557.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  weather_df['timestamp'] = pd.to_datetime(weather_df['timestamp']).astype(int) // 10**9


Merging Data Sets

In [70]:
# Leak and weather data merge
leak_data_raw['tTime'] = leak_data_raw['Time']
leak_data_raw['tTime'] = pd.to_datetime(leak_data_raw['tTime'], unit='s')
weather_df['tTime'] = pd.to_datetime(weather_df['tTime'], unit='s')

leak_data_raw['tTime'] = leak_data_raw['tTime'].dt.floor('min').dt.round('min')
df = pd.merge(leak_data_raw, weather_df, on='tTime', how='left')
df = df.drop(['temp', 'tTime'], axis=1)

In [77]:
# Normalizing leak intensity
df['leak_amount'] = 0
for index, row in leak_rate_df.iterrows():
    in_range = df['Time'].between(row['tStart'], row['tEnd'], inclusive='both')
    df.loc[in_range, 'leak_amount'] = np.array((row['LeakRate'] * (row['Duration'])), dtype=np.int32)
df['leak_amount_norm'] = (df['leak_amount'] - df['leak_amount'].min()) / (df['leak_amount'].max() - df['leak_amount'].min())